In [32]:
import pandas as pd
import numpy as np

In [33]:
df=pd.read_csv('Bacteria_dataset_Multiresictance.csv')

In [34]:
df=df.drop('Email',axis=1)

Nettoyage de la colonne Souches :

In [35]:
df['Souches']=df['Souches'].fillna('')

#Fonction qui supprime l'ID de la souche
def drop_id(souche):
  for i in range(1,len(souche)):
    if souche[i].isalpha():
      return souche[i:]
  return souche

df['Souches'] = df['Souches'].apply(drop_id)

#Standardisation de Escherichia coli
df['Souches'] = df['Souches'].replace(['E.coi','E.cli','E. coli'],'Escherichia coli')
#Standardisation de Proteus mirabilis
df['Souches'] = df['Souches'].replace(['Proeus mirabilis', 'Prot.eus mirabilis', 'Protus mirabilis'],'Proteus mirabilis')
#Standardisation de Enterobacteria spp.
df['Souches'] = df['Souches'].replace(['Enter.bacteria spp.','Enteobacteria spp.'], 'Enterobacteria spp.')
#Standardisation de Klebsiella pneumoniae
df['Souches'] = df['Souches'].replace(['Klbsiella pneumoniae', 'Klebsie.lla pneumoniae'], 'Klebsiella pneumoniae')
#Standardisation des Nan
df['Souches'] = df['Souches'].replace(['', '?','issing'], np.nan)

Les lignes qui ont le même ID_souche n'ont pas le même ID général, ni le même nom de patient... cela ressemble à des erreurs. J'ai donc supprimé cet ID qui ne correspond ni au type de bactérie, ni au nom.

Nettoyage des colonnes d'antibiotiques :

In [36]:
col_atb = ['AMX/AMP', 'AMC', 'CZ', 'FOX', 'CTX/CRO', 'IPM', 'GEN', 'AN', 'Acide nalidixique', 'ofx', 'CIP', 'C', 'Co-trimoxazole', 'Furanes','colistine']
#Première boucle pour standardiser les R, S et I
for c in col_atb:
    df[c]= df[c].str.upper().replace(['MISSING', '?'],np.nan)
    df[c] = df[c].replace('INTERMEDIATE','I')

#fonction qui transforme les R, S et I en numérique
def quantif(x):
    if x == 'R':
        return 1
    elif x == 'I':
        return 0.5
    elif x == 'S':
        return 0
    else:
        return x

#application de la fonction
df[col_atb] = df[col_atb].map(quantif)

Nettoyage des dates :

In [37]:
import re
import datetime as dt

In [38]:
#dataframe temporaire pour travailler dessus
dates_temp=df['Collection_Date']

#Etape 1 : standardiser les valeurs manquantes
dates_temp = dates_temp.replace(['error','missing','?'],np.nan)

#Etape 2 : remplacer 08/05/2021 par 2021-05-08 (8 Mai 2021, on le sait car c'est indiqué sur Kaggle)
def date_clean(d):
    try:
        if (re.search(r"(\d{4})-(\d{2})-(\d{2})",d) == None) :
            return re.sub(r"(\d{2})/(\d{2})/(\d{4})", r"\3-\2-\1", d)
        else : 
            return d
    except:
        return d
    
dates_temp =dates_temp.apply(date_clean)

#Etpe 3 : Remplacement des dernières valeurs (ont été repérées par les regex également)
dates_temp = dates_temp.replace(['5 Fev 2025','3 Jan 1019'],['2025-02-05','2019-01-03'])

#Intégration dans le df principal
df['Collection_Date'] = dates_temp

#Changement de type
df['Collection_Date'] = df['Collection_Date'].astype('datetime64[ns]')

#Pour info : repérage des valeurs pas encore transfromées entre les étapes 2 et 3:
#df_pas_nan = dates_temp.loc[dates_temp.isna()==False]
#df_pas_nan.loc[df_pas_nan.apply(lambda x:re.search(r"(\d{4})-(\d{2})-(\d{2})",x) == None)].unique()

#Et à la fin, pour vérifier que le mois est bien à sa place, on doit obtenir 2 en premier
#df['Collection_Date'].dt.month

Nettoyage Adresses :

In [39]:

# === Dictionnaire abréviation → nom complet ===
etat_noms_complets = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

# === Fonctions d'extraction ===

# Extraire État et Code Postal
def extraire_etat_cp(adresse):
    match = re.search(r',\s*([A-Z]{2})\s+(\d{5})$', adresse)
    if match:
        etat = match.group(1)
        code_postal = match.group(2)
        return pd.Series([etat, code_postal])
    return pd.Series(['', ''])

# Convertir abréviation en nom complet
def etat_nom_complet(abbr):
    return etat_noms_complets.get(abbr, '')

# Déduire le pays à partir de l’État
def etat_vers_pays(etat):
    return 'United States' if etat in etat_noms_complets else ''

# Extraire la ville approximative
def extraire_ville(adresse):
    match = re.search(r',\s*(.*?)\s*,\s*[A-Z]{2}\s+\d{5}$', adresse)
    return match.group(1) if match else ''

# === Application des fonctions ===
df[['Etat', 'Code_Postal']] = df['Address'].apply(extraire_etat_cp)
df['Etat_Complet'] = df['Etat'].apply(etat_nom_complet)
df['Pays'] = df['Etat'].apply(etat_vers_pays)
df['Ville'] = df['Address'].apply(extraire_ville)

Nettoyage Facteurs risque et Infection Freq :

In [40]:
df['Diabetes'] = df['Diabetes'].replace(['No','True','?'],[False, True, np.nan])
#df['Diabetes'] = df['Diabetes'].replace('True',True)
#df['Diabetes'] = df['Diabetes'].replace('?',np.nan)
df['Diabetes'] = df['Diabetes'].replace('missing',np.nan)
df['Hypertension'] = df['Hypertension'].replace('No',False)
df['Hypertension'] = df['Hypertension'].replace('Yes',True)
df['Hypertension'] = df['Hypertension'].replace('?',np.nan)
df['Hypertension'] = df['Hypertension'].replace('missing',np.nan)
df['Hospital_before'] = df['Hospital_before'].replace('No',False)
df['Hospital_before'] = df['Hospital_before'].replace('Yes',True)
df['Hospital_before'] = df['Hospital_before'].replace('?',np.nan)
df['Hospital_before'] = df['Hospital_before'].replace('missing',np.nan)
df['Infection_Freq'] = pd.to_numeric(df['Infection_Freq'], errors='coerce').astype('Int64')


Nettoyage age, genre et name :

In [41]:
# 2. Transformer 'Name' to 'ID Name' avec numero unique
if 'Name' in df.columns:
    name_to_id = {name: i for i, name in enumerate(df['Name'].unique())}
    df['Name'] = df['Name'].map(name_to_id)
    df = df.rename(columns={'Name': 'ID Name'})
    print("Column 'Name' transformed to 'ID Name' and unique IDs assigned.")
else:
    print("Column 'Name' not found. It might have already been transformed to 'ID Name'.")

# 4. Séparer 'age/gender' et faire deux colonnes
if 'age/gender' in df.columns:
    df[['age', 'gender']] = df['age/gender'].str.split('/', expand=True)
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    df = df.drop(columns=['age/gender'])
    print("Column 'age/gender' successfully split into 'age' and 'gender'.")
else:
    print("Column 'age/gender' not found in the DataFrame.")

# 5. Convertir le type colonne 'age' 
if 'age' in df.columns:
    df['age'] = df['age'].astype('Int64')
    print("Column 'age' successfully converted to integer type (Int64).")
else:
    print("Column 'age' not found in the DataFrame.")

#df = df[final_column_order]
#print("Columns 'age' and 'gender' reordered after 'ID Name'.")


Column 'Name' transformed to 'ID Name' and unique IDs assigned.
Column 'age/gender' successfully split into 'age' and 'gender'.
Column 'age' successfully converted to integer type (Int64).


In [42]:
df = df.drop(['Notes', 'Address', 'Etat', 'Code_Postal'],axis=1)
df.head()

,ID,ID Name,Souches,Diabetes,Hypertension,Hospital_before,Infection_Freq,AMX/AMP,AMC,CZ,...,C,Co-trimoxazole,Furanes,colistine,Collection_Date,Etat_Complet,Pays,Ville,age,gender
0,S290,0,Escherichia coli,False,False,False,0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,2025-02-05,Rhode Island,United States,Paulfurt,37,F
1,S291,1,Morganella morganii,True,False,False,3,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2021-05-08,Hawaii,United States,South Tanyatown,29,F
2,S292,2,Proteus mirabilis,True,False,False,3,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,2022-05-01,Hawaii,United States,North Benjamin,77,F
3,S293,3,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2020-01-05,Illinois,United States,Andrewbury,<NA>,NaN
4,S294,4,Escherichia coli,False,True,False,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaT,Maine,United States,Torresmouth,13,F


In [43]:
#On enlève les lignes dont la souche est absente
df = df.loc[df['Souches'].isna() == False]

In [44]:
#Changement des types
df[['Diabetes','Hypertension','Hospital_before']] = df[['Diabetes','Hypertension','Hospital_before']].astype('boolean')
df['ID Name'] = df['ID Name'].astype('object')

In [45]:
df.to_csv("Biomedic_clean.csv",index=False)